# Sentence Transformer: Trained on Binary Citation Prediction

- TODO Train on OCB+WS data


In [1]:
!which python

/home/mostendorff/miniconda2/envs/legal-docsim/bin/python


In [2]:
!pip freeze|grep transformers

sentence-transformers==0.4.1.2
transformers==4.2.0


In [3]:
import re
import json
import os
import requests
import pickle
import pandas as pd
import logging
from pathlib import Path
from collections import defaultdict
from docsim.gold_standard import GoldStandard
from smart_open import open
import random
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm
from IPython.core.display import display

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from gensim.models import KeyedVectors

from docsim.methods import RecSys
from docsim.methods.keyed_vector_based import KeyedVectorRecSys, MultiKeyedVectorRecSys, EnsembleKeyedVectorRecSys
from docsim.methods.text_based import TfIdfRecSys, Doc2VecRecSys, WeightedAvgWordVectorsRecSys
from docsim.methods.transformer_based import TransformerRecSys
from docsim.methods.graph_based import GraphEmbeddingRecSys
from docsim.environment import get_env
from docsim.experiment import Experiment
from utils import get_mean_avg_precision, get_avg_precision, highlight_max


logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)



unable to import 'smart_open.gcs', disabling that module


In [4]:
!nvidia-smi

Wed Feb 10 19:28:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.31       Driver Version: 440.31       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:1B:00.0 Off |                  N/A |
| 27%   32C    P8    12W / 250W |      0MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 6000     On   | 00000000:1C:00.0 Off |                  Off |
| 33%   28C    P8     4W / 260W |      0MiB / 24220MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  Q

In [5]:
os.environ['CUDA_VISIBLE_DEVICES'] = '6'
#os.environ['CUDA_VISIBLE_DEVICES'] = '1,2,3,5,6,7'

In [6]:
from sentence_transformers.models import Pooling, Transformer
from smart_open import open
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, losses

In [10]:
train_batch_size = 12

data_dir = './data'

experiment = 'ocb'
experiment = 'wikisource'


models_dir = '/data/datasets/BERT_pre_trained_models/pytorch'

model_name = 'aueb-legal-bert-base-uncased'
max_token_length = 512

#model_name = 'legal-bert'
#model_name = 'longformer-large-4096'
#
#model_name = 'longformer-base-4096'

if 'longformer-large' in model_name:
    train_batch_size = 4
    max_token_length = 4096
    
if 'longformer-base' in model_name:
    train_batch_size = 2
    max_token_length = 3500 # 2048
    

train_epochs = 3

#model_output_dir = f'./models/{experiment}/sentence_joint_{model_name}'
model_output_dir = f'./models/{experiment}/sentence_{model_name}'


model_name_or_path = os.path.join(models_dir, model_name)
model_name_or_path

'/data/datasets/BERT_pre_trained_models/pytorch/aueb-legal-bert-base-uncased'

In [11]:
max_token_length

512

In [12]:
model_output_dir

'./models/wikisource/sentence_aueb-legal-bert-base-uncased'

In [13]:
train_batch_size

12

In [14]:
# Sentence transformer
word_embedding_model = Transformer(model_name_or_path, max_seq_length=max_token_length)
pooling_model = Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


In [15]:
import torch

from transformers import BertTokenizerFast

#tokenizer = BertTokenizerFast.from_pretrained(model_name_or_path)
tokenizer = None

In [16]:
from pathlib import Path
from gensim.models import KeyedVectors

from docsim.environment import get_env
from docsim.experiment import Experiment
from docsim.methods.graph_based import GraphEmbeddingRecSys
from docsim.methods.text_based import TfIdfRecSys, Doc2VecRecSys, WeightedAvgWordVectorsRecSys
from docsim.methods.transformer_based import TransformerRecSys, SentenceTransformerRecSys


env = get_env()
data_dir = Path(data_dir)


/data/experiments/mostendorff/legal-docsim/environments
Environment detected: gpu_server2 (in default.yml)


## Train on OCB+WS

In [17]:
from sentence_transformers import SentenceTransformer, InputExample

pos_label = 1.
neg_label = 0.


# Sentence transformer data
train_examples = []
exps = [
    #'ocb',
    'wikisource'
]

for experiment in exps:    
    logger.info(f'Experiment: {experiment}')
    exp = Experiment(name=experiment, env=env, data_dir=data_dir)
    exp.load_data()
    exp.filter_docs()

    # Extract positive samples based on citations
    not_found = 0
    found = 0
    cit_ids = set()

    pos_from_texts = []
    pos_to_texts = []

    for cit in exp.cits:
        from_id, to_id = cit

        if from_id in exp.doc_id2idx and to_id in exp.doc_id2idx:
            cit_id = tuple(sorted([from_id, to_id]))
            cit_ids.add(cit_id)

            pos_from_texts.append(exp.texts[exp.doc_id2idx[from_id]])
            pos_to_texts.append(exp.texts[exp.doc_id2idx[to_id]]) 
            found += 1
        else:
            not_found += 1

    logger.info(f'Citing docs - Not found: {not_found / len(exp.cits)}; Found: {found / len(exp.cits)}')

    # negative samples
    neg_ratio = 1.0
    neg_needed = len(pos_from_texts) * neg_ratio

    logger.info(f'Negatives needed: {neg_needed}')

    neg_from_texts = []
    neg_to_texts = []

    while len(neg_from_texts) < neg_needed:
        # random samples
        from_id = random.choice(exp.idx2doc_id)
        to_id = random.choice(exp.idx2doc_id)

        if from_id == to_id:
            continue

        cit_id = tuple(sorted([from_id, to_id]))

        if cit_id in cit_ids:
            continue

        # add
        cit_ids.add(cit_id)
        neg_from_texts.append(exp.texts[exp.doc_id2idx[from_id]])
        neg_to_texts.append(exp.texts[exp.doc_id2idx[to_id]])     
    
    # convert to st-input examples
    for a, b in zip(pos_from_texts, pos_to_texts):
        train_examples.append(
            InputExample(texts=[a, b], label=pos_label)
        )                

    for a, b in zip(neg_from_texts, neg_to_texts):
        train_examples.append(
            InputExample(texts=[a, b], label=neg_label)
        )
    
    logger.info(f'Experiment loaded')
    
logger.info(f'Train on {len(train_examples):,}')
    

INFO:__main__:Experiment: wikisource
INFO:docsim.experiment:Documents after filtering: 1,378 (before 64,003)
INFO:__main__:Citing docs - Not found: 0.9904856137895252; Found: 0.009514386210474875
INFO:__main__:Negatives needed: 3154.0
INFO:__main__:Experiment loaded
INFO:__main__:Train on 6,308


In [21]:
from pathlib import Path
from gensim.models import KeyedVectors

from docsim.environment import get_env
from docsim.experiment import Experiment
from docsim.methods.graph_based import GraphEmbeddingRecSys
from docsim.methods.text_based import TfIdfRecSys, Doc2VecRecSys, WeightedAvgWordVectorsRecSys
from docsim.methods.transformer_based import TransformerRecSys, SentenceTransformerRecSys


env = get_env()
data_dir = Path(data_dir)

logger.info(f'Experiment: {experiment}')

exp = Experiment(name=experiment, env=env, data_dir=data_dir)

exp.load_data()
exp.filter_docs()

INFO:__main__:Experiment: ocb


/data/experiments/mostendorff/legal-docsim/environments
Environment detected: gpu_server2 (in default.yml)


INFO:docsim.experiment:Documents loaded: 1,632
INFO:docsim.experiment:Unique documents in gold standard: 1,623
INFO:docsim.experiment:Documents after filtering: 1,590 (before 1,632)


In [14]:
texts = exp.texts
cits = exp.cits
doc_id2idx = exp.doc_id2idx
idx2doc_id = exp.idx2doc_id

len(exp.texts)

1590

In [15]:
len(exp.cits)

70865

In [16]:
exp.cits[:3]

[['1994907', '197 N. Y. 541'],
 ['7797724', '960 F.2d 1217'],
 ['4334708', '315 U. S. 568']]

In [17]:
from torch.utils.data import Dataset
from sentence_transformers import SentenceTransformer, InputExample

class CitedDocumentsDataset(Dataset):
    pos_label = 1.
    neg_label = 0.
    
    def __init__(self, exp, tokenizer, return_labels=True, sample_n=0, max_length=512):
        
        self.texts = exp.texts
        self.cits = exp.cits
        self.doc_id2idx = exp.doc_id2idx
        self.idx2doc_id = exp.idx2doc_id
        
        self.tokenizer = tokenizer
        self.return_labels = return_labels
        self.sample_n = sample_n
        self.inputs = []
        self.samples = []
        self.max_length = max_length
    
    def load(self, tokenize=True):
        logger.info(f'Corpus size: {len(self.doc_id2idx)}')

        # Extract positive samples based on citations
        not_found = 0
        found = 0
        cit_ids = set()

        pos_from_texts = []
        pos_to_texts = []

        for cit in self.cits:

            from_id, to_id = cit

            if from_id in self.doc_id2idx and to_id in self.doc_id2idx:
                cit_id = tuple(sorted([from_id, to_id]))
                cit_ids.add(cit_id)

                pos_from_texts.append(self.texts[self.doc_id2idx[from_id]])
                pos_to_texts.append(self.texts[self.doc_id2idx[to_id]])        

                found += 1
            else:
                #raise ValueError(cit)
                not_found += 1

        logger.info(f'Citing docs - Not found: {not_found / len(self.cits)}; Found: {found / len(self.cits)}')


        # negative samples
        neg_ratio = 1.0
        neg_needed = len(pos_from_texts) * neg_ratio

        logger.info(f'Negatives needed: {neg_needed}')

        neg_from_texts = []
        neg_to_texts = []

        while len(neg_from_texts) < neg_needed:
            # random samples
            from_id = random.choice(self.idx2doc_id)
            to_id = random.choice(self.idx2doc_id)

            if from_id == to_id:
                continue

            cit_id = tuple(sorted([from_id, to_id]))

            if cit_id in cit_ids:
                continue

            # add
            cit_ids.add(cit_id)
            neg_from_texts.append(self.texts[self.doc_id2idx[from_id]])
            neg_to_texts.append(self.texts[self.doc_id2idx[to_id]])     

        if tokenize:            
            logger.info(f'Tokenize... {len(pos_from_texts):,} + {len(neg_from_texts):,}  from texts')

            self.from_inputs = self.tokenizer(
                text=pos_from_texts + neg_from_texts,
                add_special_tokens=False,
                return_attention_mask=False,
                return_tensors='pt',
                padding='max_length',
                max_length=self.max_length,
                truncation=True,
                return_token_type_ids=False,
            )

            logger.info(f'Tokenize... {len(pos_to_texts):,} + {len(neg_to_texts):,} to texts')

            self.to_inputs = self.tokenizer(
                text=pos_to_texts + neg_to_texts,
                add_special_tokens=False,
                return_attention_mask=False,
                return_tensors='pt',
                padding='max_length',
                max_length=self.max_length,
                truncation=True,
                return_token_type_ids=False,
            )

            if self.return_labels:
                labels = [self.pos_label] * len(pos_from_texts) + [self.neg_label] * len(neg_from_texts) 

                self.labels = torch.tensor(labels)

            logger.info('Dataset loaded')
            
            return self.from_inputs, self.to_inputs, self.labels
            
        else:
            # return input examples -> sentence transformer inputs
            input_examples = []
            
            for a, b in zip(pos_from_texts, pos_to_texts):
                input_examples.append(
                    InputExample(texts=[a, b], label=self.pos_label)
                )                
            
            for a, b in zip(neg_from_texts, neg_to_texts):
                input_examples.append(
                    InputExample(texts=[a, b], label=self.neg_label)
                )
                
            return input_examples
            
            
    def __getitem__(self, idx):
        return [
            self.from_inputs['input_ids'][idx], 
            self.to_inputs['input_ids'][idx]
        ], self.labels[idx]
        #return {k: v[idx] for k, v in self.inputs.items()}

    def __len__(self):
        return len(self.from_inputs['input_ids'])

In [18]:
train_ds = CitedDocumentsDataset(exp, tokenizer, max_length=max_token_length)

In [19]:
#train_ds.load(tokenize=True)

In [20]:
train_examples = train_ds.load(tokenize=False)

INFO:__main__:Corpus size: 1590
INFO:__main__:Citing docs - Not found: 0.9772525224017498; Found: 0.022747477598250194
INFO:__main__:Negatives needed: 1612.0


In [18]:
from torch.utils.data import DataLoader

train_dl = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)

In [19]:
train_loss = losses.CosineSimilarityLoss(model)

In [20]:
# Sample of gold standard
gs_df = exp.gs.df.sample(n=100)
gs_df

,seed_id,target_id,label
37860,86931,93489,United States Supreme Court decisions on treaties
173518,86725,85334,United States Supreme Court decisions on evidence
6851,99561,85214,United States Supreme Court decisions on treaties
156242,96786,101911,United States Supreme Court decisions on the F...
187363,112513,112464,United States Supreme Court decisions on the F...
...,...,...,...
117259,90228,89526,United States Supreme Court decisions on secur...
12898,89181,89949,United States Supreme Court decisions on treaties
44444,88305,97934,United States Supreme Court decisions on treaties
182609,88112,88493,United States Supreme Court decisions on civil...


In [21]:
from sentence_transformers.evaluation.BinaryClassificationEvaluator import BinaryClassificationEvaluator

# Evaluator

from_texts = []
to_texts = []

for from_id, to_id in gs_df[['seed_id', 'target_id']].values:
    if from_id in exp.doc_id2idx and to_id in exp.doc_id2idx:
        from_texts.append(exp.texts[exp.doc_id2idx[from_id]])
        to_texts.append(exp.texts[exp.doc_id2idx[to_id]])
        
labels = [1] * len(from_texts)

test_limit = 10

evaluator = BinaryClassificationEvaluator(from_texts[:test_limit], to_texts[:test_limit], labels[:test_limit])


## Train

In [23]:
# Training
model.fit(
    train_objectives=[(train_dl, train_loss)],
    epochs=train_epochs, # try 1-4
    warmup_steps=100,
    #evaluator=evaluator,
    #evaluation_steps=evaluation_steps,  # increase to 5000 (full dataset => 20k steps)
    output_path=model_output_dir,
    #output_path_ignore_not_empty=True
)


In [24]:
model.save(model_output_dir)

INFO:sentence_transformers.SentenceTransformer:Save model to ./models/wikisource/sentence_aueb-legal-bert-base-uncased


In [25]:
model_output_dir

'./models/wikisource/sentence_aueb-legal-bert-base-uncased'